In [12]:

import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [11]:

# Celda 2: Descargar recursos de NLTK
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords', quiet=True)
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet', quiet=True)


In [8]:

class MovieInformationRetrieval:
    def __init__(self, movies_path, reviews_path):
        # Cargar datos
        self.movies_df = pd.read_csv(movies_path)
        self.reviews_df = pd.read_csv(reviews_path)

        # Convertir todas las columnas relevantes a string
        self.reviews_df['review_content'] = self.reviews_df['review_content'].fillna('').astype(str)

        # Configurar herramientas de procesamiento de texto
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()

        # Preprocesar reseñas de críticos
        self.preprocessed_corpus = self.preprocess_reviews()

        # Vectorizar las reseñas preprocesadas
        self.vectorizer = TfidfVectorizer()
        self.tfidf_matrix = self.vectorizer.fit_transform(self.preprocessed_corpus)

    def clean_text(self, text):
        # Manejar valores NaN y convertir a cadena
        if pd.isna(text) or text is None:
            return ""

        # Convertir a cadena y minúsculas
        text = str(text).lower()

        # Eliminar puntuación
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Eliminar caracteres especiales
        text = re.sub(r'[^a-zA-Z\s]', '', text)

        return text

    def preprocess_text(self, text):
        # Limpiar texto
        text = self.clean_text(text)

        # Tokenización
        tokens = word_tokenize(text)

        # Eliminar stopwords
        tokens = [token for token in tokens if token not in self.stop_words]

        # Lematización
        tokens = [self.lemmatizer.lemmatize(token) for token in tokens]

        # Normalización: unir tokens
        return ' '.join(tokens)

    def preprocess_reviews(self):
        # Convertir columnas a string y reemplazar NaN
        columns_to_clean = [
            'movie_title', 'movie_info', 'genres',
            'directors', 'actors'
        ]
        for col in columns_to_clean:
            self.movies_df[col] = self.movies_df[col].fillna('').astype(str)

        # Agrupar reseñas por película usando el rotten_tomatoes_link
        grouped_reviews = self.reviews_df.groupby('rotten_tomatoes_link')['review_content'].apply(lambda x: ' '.join(x.astype(str))).reset_index()

        # Combinar con información de la película
        merged_corpus = []
        for _, movie in self.movies_df.iterrows():
            # Buscar reseñas correspondientes
            review_text = grouped_reviews[grouped_reviews['rotten_tomatoes_link'] == movie['rotten_tomatoes_link']]['review_content'].values

            # Combinar toda la información textual
            full_text = ' '.join([
                movie['movie_title'],
                movie['movie_info'],
                movie['genres'],
                movie['directors'],
                movie['actors'],
                review_text[0] if len(review_text) > 0 else ''
            ])

            # Preprocesar texto combinado
            merged_corpus.append(self.preprocess_text(full_text))

        return merged_corpus

    def search(self, query, top_k=5):
        # Preprocesar consulta
        processed_query = self.preprocess_text(query)

        # Vectorizar consulta
        query_vector = self.vectorizer.transform([processed_query])

        # Calcular similitud coseno
        cosine_similarities = cosine_similarity(query_vector, self.tfidf_matrix).flatten()

        # Obtener índices de las películas más relevantes
        top_indices = cosine_similarities.argsort()[::-1][:top_k]

        # Crear DataFrame de resultados
        results = self.movies_df.iloc[top_indices].copy()
        results['relevancia'] = cosine_similarities[top_indices]

        return results

    def print_results(self, query, results):
        print(f"--- Resultados para: '{query}' ---\n")

        for idx, row in results.iterrows():
            print(f"Película {idx+1} (Relevancia: {row['relevancia']:.4f}):")
            print(f"Título: {row['movie_title']}")
            print(f"Género: {row['genres']}")
            print(f"Descripción: {row['movie_info']}")
            print(f"Director: {row['directors']}")
            print(f"Actores: {row['actors']}")
            print(f"Fecha de Lanzamiento: {row['original_release_date']}")
            print('-' * 50 + '\n')


In [9]:

#  Inicialización del sistema de recuperación
retrieval_system = MovieInformationRetrieval(
    movies_path='rotten_tomatoes_movies.csv',
    reviews_path='rotten_tomatoes_critic_reviews.csv'
)


In [10]:

while True:
    # Solicitar consulta al usuario
    query = input("\n--- Sistema de Búsqueda de Películas ---\n"
                  "Ingrese su consulta (o 'salir' para terminar): ")

    # Opción de salida
    if query.lower() == 'salir':
        print("Gracias por usar el sistema de búsqueda de películas. ¡Hasta luego!")
        break

    # Validar que la consulta no esté vacía
    if not query.strip():
        print("Por favor, ingrese una consulta válida.")
        continue

    try:
        # Realizar búsqueda
        results = retrieval_system.search(query)

        # Mostrar resultados
        retrieval_system.print_results(query, results)

    except Exception as e:
        print(f"Ocurrió un error al procesar su búsqueda: {e}")


--- Sistema de Búsqueda de Películas ---
Ingrese su consulta (o 'salir' para terminar): space movies
--- Resultados para: 'space movies' ---

Película 9745 (Relevancia: 0.5421):
Título: Lost in Space
Género: Action & Adventure, Science Fiction & Fantasy
Descripción: The space family Robinson is on a journey to Alpha Prime in the hopes of establishing a colony there and thereby saving humanity from extinction. Their plans are foiled by the evil Dr. Smith (Gary Oldman) and they find themselves curiously enough, lost in space.
Director: Stephen Hopkins
Actores: William Hurt, Matt LeBlanc, Gary Oldman, Mimi Rogers, Heather Graham, Lacey Chabert, Jack Johnson (II), Jared Harris, Mark Goddard, Lennie James, Marta Kristen, June Lockhart, Edward Fox, Adam Sims, Angela Cartwright, Dick Tufeld
Fecha de Lanzamiento: 1998-04-03
--------------------------------------------------

Película 15838 (Relevancia: 0.4828):
Título: The Space Between Us
Género: Action & Adventure, Science Fiction & Fantasy